# 9장 컴퓨터 비전을 위한 고급 딥러닝 2부

**감사말**: 프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 9장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

**tensorflow 버전과 GPU 확인**
- 구글 코랩 설정: '런타임 -> 런타임 유형 변경' 메뉴에서 GPU 지정 후 아래 명령어 실행 결과 확인

    ```
    !nvidia-smi
    ```

- 사용되는 tensorflow 버전 확인

    ```python
    import tensorflow as tf
    tf.__version__
    ```
- tensorflow가 GPU를 사용하는지 여부 확인

    ```python
    tf.config.list_physical_devices('GPU')
    ```

## 9.3 합성곱 신경망 기본 아키텍처

**모델 아키텍처**는 모델 설계방식을 의미하며
딥러닝 모델을 구성할 때 매우 중요하다.
주어진 데이터셋과 해결해야 문제에 따라 적절한 층을 적절하게 구성해서
모델을 구현해야 한다.
좋은 모델 아키텍처를 사용할 수록 적은 양의 데이터을 이용하여 보다 빠르게
좋은 성능의 모델을 얻을 가능성이 높아진다. 
아쉽게도 좋은 모델 아키텍처와 관련된 이론은 없으며
많은 경험을 통한 직관이 보다 중요한 역할을 수행한다. 

여기서는 실전에서 최고 성능을 발휘한 합성곱 신경망 몇 개를 이용하여
주요 합성곱 신경망 모델의 기본이 되는 아키텍처 3 개를 살펴본다.

- 잔차 연결(residual connections)
- 배치 정규화(batch normalization)
- 채널 분리 합성곱(depthwise separable convolutions)

### 9.3.1 모듈, 계층, 재활용

모든 유명한 합성곱 신경망 모델은 **모듈(블록)**을 **계층**적으로 쌓아 올린 구조를 갖는다.
여기서 모듈(블록)은 여러 개의 층(레이어)으로 구성되며, 하나의 모듈(블록)이 여러 번 **재활용**되기도 한다. 
예를 들어, 8장에서 다룬 VGG16 모델은 "Conv2D, Conv2D, MaxPooling2D" 로 구성된 모듈(블록)을 
계층적으로 구성하였다. 

대부분의 합성곱 신경망 모델의 또다른 특징는 **특성 피라미드** 형식의 계층적 구조를 사용하는 점이다.
VGG16의 경우에 필터 수를 32, 64, 128 등으로 수를 늘리는 반면에 특성맵(feature maps)의 
크기는 그에 상응하게 줄여 나간다(아래 그림 참조).

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/09-08.png" style="width:80%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

**좁은 층 깊은 신경망 모델**

일반적으로 많은 유닛을 사용하는 넓은 층을 몇 게 쌓는 것보다
적은 유닛을 사용하는 층을 많이 쌓을 때 모델의 성능이 좋아진다. 
하지만 층을 많이 쌓을 수록 
전달되어야 하는 그레이디언트가 사라지는 문제(vanishing gradient problem)가 
발생하여 이를 극복해야 하는 아키텍처(설계방식)을 사용해야 한다.
이를 위한 대표적인 아키텍처는 **잔차 연결**(residual connections)이다.

### 9.3.2 잔차 연결

하나 또는 여러 개의 층으로 구성된 블록(모듈)의 입력값을
블록을 통과하여 생성된 출력값과 합쳐서 다음 블록으로 전달하는 아키텍처이다.
이 방식을 통해 블록의 입력값에 대한 정보가 보다 정확하게 상위 블록에 전달된다.
잔차 연결 아키텍처는 2015년에 소개된 ResNet 계열의 모델에서 처음 사용되었으며,
2015년 ILSVRC 이미지 분류 경진대회에서 1등을 차지했다.
실제로 잔차 연결을 이용하면 아무리 블록을 임의로 많이 쌓아도 모델 훈련에 전혀 문제가 없다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/09-09.png" style="width:30%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

**모양(shape) 맞추기**

잔차 연결을 사용할 때 기본적을로 주의해야할 사항은 두 텐서의 모양을 맞추는 일이다.
블록을 통과한 출력값과 블록을 건너 뛴 입력값의 모양이 동일해야 하기 때문이다. 

- 맥스풀링을 사용하지 않는 경우
    - `Conv2D` 층: `padding="same"` 옵션을 사용하여 모양을 유지
    - 필터 수가 변하는 경우: 잔차에 `Conv2D` 층을 이용하여 필터 수를 맞춤. 필터 크기는 `1x1` 사용.
        활성화 함수는 없음.

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)

residual = x

x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)  # padding 사용

residual = layers.Conv2D(64, 1)(residual)                       # 필터 수 맞추기

x = layers.add([x, residual])

- 맥스풀링을 사용하는 경우
    - 잔차에 `Conv2D` 층을 적용할 때 보폭 사용

In [0]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)

residual = x

x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)                   # 맥스풀링

residual = layers.Conv2D(64, 1, strides=2)(residual)            # 보폭 사용

x = layers.add([x, residual])

**예제**

In [0]:
# 입력층
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

# 은닉층
def residual_block(x, filters, pooling=False):
    residual = x
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    if pooling:                          # 맥스풀링 사용하는 경우
        x = layers.MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]:  # 필터 수가 변하는 경우
        residual = layers.Conv2D(filters, 1)(residual)
    x = layers.add([x, residual])
    
    return x

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = layers.GlobalAveragePooling2D()(x)

# 출력층
outputs = layers.Dense(1, activation="sigmoid")(x)

# 모델 설정
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

### 9.3.3 배치 정규화

### 9.3.4 채널 분리 합성곱

### 9.3.5 미니 Xception 유형 모델

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
!unzip -qq train.zip

In [0]:
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

In [0]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [0]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(
        size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset)